In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
%%time
train_metadata_kaggle = dd.read_csv('mydata_train_metadata.csv')
test_metadata_kaggle = dd.read_csv('mydata_test_metadata.csv')
train_metadata_kaggle = train_metadata_kaggle.compute()
test_metadata_kaggle = test_metadata_kaggle.compute()
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 104) (3492890, 103)
CPU times: user 2min 30s, sys: 9.12 s, total: 2min 39s
Wall time: 28.5 s


In [5]:
%%time
train_metadata = dd.read_csv('train_metadata_final.csv')
test_metadata = dd.read_csv('test_metadata_final.csv')
train_metadata = train_metadata.compute()
test_metadata = test_metadata.compute()
print(train_metadata.shape,test_metadata.shape)

(7848, 233) (3492890, 232)
CPU times: user 3min 42s, sys: 13.1 s, total: 3min 55s
Wall time: 43.6 s


In [6]:
%%time
test_metadata_kaggle = test_metadata_kaggle.reset_index(drop=True)
test_metadata = test_metadata.reset_index(drop=True)

CPU times: user 5.91 s, sys: 8.28 s, total: 14.2 s
Wall time: 14.2 s


In [7]:
print(train_metadata_kaggle['object_id'].equals(train_metadata['object_id']))

True


In [8]:
print(test_metadata_kaggle['object_id'].equals(test_metadata['object_id']))

True


In [9]:
temp_columns = ['object_id','A0_max_flux','A0_min_flux','A0_mean_flux','A0_median_flux','A0_std_flux',
'A1_max_flux','A1_min_flux','A1_mean_flux','A1_median_flux','A1_std_flux',
'A2_max_flux','A2_min_flux','A2_mean_flux','A2_median_flux','A2_std_flux',
'A3_max_flux','A3_min_flux','A3_mean_flux','A3_median_flux','A3_std_flux',
'A4_max_flux','A4_min_flux','A4_mean_flux','A4_median_flux','A4_std_flux',
'A5_max_flux','A5_min_flux','A5_mean_flux','A5_median_flux','A5_std_flux']

In [11]:
test_metadata.drop([x for x in test_metadata.columns if x not in temp_columns], axis=1,inplace=True)
gc.collect()

574

In [12]:
test_metadata_kaggle.tail()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF,A1_minus_3_sigma,A5_max_median_diff_flux,A5_minus_3_sigma,A5_max_mean_diff_flux
3492885,130787966,-49.490440,75.235832,3.905430,1.703813,21.009541,0.625654,1.341227,70.066055,14.846084,10.951475,12.628468,1.307980,0.013793,374.420803,8.678639,5.954086e+03,7.560569,15.902124,124.726272,31.936629,7.843372,35.655183,32.200453,0.154555,-0.174445,23.565507,13.595040,6.803259,2.380728,63.739853,39.279845,3.023876,1.647382,82.545904,81.181195,0.670890,0.882368,187.846479,74.176357,3.792912,1.576256,172.934431,184.079710,-0.297825,0.291885,145.0,8.0,10.0,20.061909,0.151706,4.0,43.0,89.761400,89.761400,89.7614,0.4493,0.9954,41.9836,0.036,1.937777,2.277911,1.215719,35.655180,2.377012,0.0000,0.0909,0.0,0.0,0.000,0.00000,0.5557,-2.620740,0.0,5.474970e-13,0.103244,2.16581,0.050748,0.683935,2.26490,0.829141,2.439200,2.16581,0.500000,0.953504,0.808853,0.966782,2.32229,0.185745,19.47180,0.427032,10.79380,0.000000,1.000000,1.000000,0,0,0,0,-20.260712,72.660479,-92.906480,71.556374
3492886,130787971,-58.301781,59.105938,2.910128,1.390304,18.432931,-0.033138,1.598772,70.469521,14.704315,10.958591,12.573206,1.570236,0.014286,772.852614,9.258595,2.735284e+04,9.229165,35.392055,117.407719,40.344517,3.317347,47.270954,21.592998,-1.528033,0.106061,23.940401,25.706965,1.965597,0.964946,106.555644,97.317107,4.782199,2.138468,13.695812,30.341361,-0.615031,-0.353402,198.983121,128.149311,1.826418,0.809600,111.513930,121.261346,-0.765793,-0.401619,140.0,6.0,10.0,16.961143,-0.250912,5.0,95.0,0.977400,0.977400,0.9774,0.6729,0.0614,43.0419,0.083,2.063756,0.297612,0.715511,-47.270954,-3.376497,0.0000,0.0000,2.0,0.0,0.000,0.00000,0.6960,-13.268991,0.0,8.997960e-14,0.009136,3.61916,-0.781019,0.920752,2.63034,0.692992,0.862991,3.61916,0.652174,0.912471,0.714979,-0.248684,1.33370,0.073276,16.81010,0.711031,28.97450,0.000000,1.000000,1.000000,0,0,0,0,-14.522348,36.030482,-83.425852,43.320908
3492887,130787974,-79.991745,341.

In [13]:
test_metadata.tail()

,object_id,A0_min_flux,A0_max_flux,A0_std_flux,A0_mean_flux,A0_median_flux,A1_min_flux,A1_max_flux,A1_std_flux,A1_mean_flux,A1_median_flux,A2_min_flux,A2_max_flux,A2_std_flux,A2_mean_flux,A2_median_flux,A3_min_flux,A3_max_flux,A3_std_flux,A3_mean_flux,A3_median_flux,A4_min_flux,A4_max_flux,A4_std_flux,A4_mean_flux,A4_median_flux,A5_min_flux,A5_max_flux,A5_std_flux,A5_mean_flux,A5_median_flux
3492885,130787966,-19.231127,18.327864,10.015860,2.377012,0.194802,-5.105165,22.772732,7.467904,2.143,0.8790,-2.620740,18.966825,5.440,3.541103,2.189612,-8.542406,24.314121,8.09,3.588952,1.232520,-23.242554,73.366840,19.793636,6.059564,3.450523,-49.490440,75.235830,32.195312,3.679456,2.575351
3492886,130787971,-13.702611,5.737575,6.879981,-3.376497,-5.786102,-6.108143,14.889024,5.566116,2.176,1.1770,-13.268991,44.679916,13.820,4.632854,1.252785,-11.002852,9.428670,6.11,0.622537,0.740433,-27.026964,59.105938,17.087936,6.418810,4.378683,-58.301780,46.180240,28.761728,2.859332,10.149758
3492887,130787974,-19.901522,117.104140,33.753580,11.206562,5.168170,-3.021157,65.796760,25.748106,14.120,2.7200,-4.419514,331.153630,117.700,79.828990,5.408035,-7.961402,308.078770,92.80,54.059563,3.760870,-15.829834,341.318900,122.374030,64.385560,4.701163,-79.991745,307.034970,90.453160,26.689178,-4.267330
3492888,130788053,-9.348281,72.267590,23.513706,6.616929,0.730023,-1.932591,66.417595,24.979921,9.860,0.5176,-10.273894,5.598794,3.746,-0.260037,-0.425032,-11.894145,17.121632,7.68,1.022865,1.275511,-29.653970,17.978836,11.335240,0.889137,0.173198,-31.093233,47.343845,17.408049,4.865474,2.143283
3492889,130788054,-13.072737,30.335392,11.523481,0.690040,-1.810264,-3.566863,47.519875,12.820799,3.791,0.1388,-5.267357,27.842110,7.820,2.587629,0.787241,-6.765305,25.873100,9.10,4.772007,2.447138,-12.543804,24.374737,9.450494,5.907181,4.280222,-29.865211,92.339670,32.776836,17.445766,6.951662


In [14]:
train_metadata_kaggle.tail()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF,A1_minus_3_sigma,A5_max_median_diff_flux,A5_minus_3_sigma,A5_max_mean_diff_flux
7843,130739978,-105.375282,517.602478,6.786007,1.599034,50.600944,7.259594,1.700991,72.230759,16.391897,11.132936,14.645405,1.401451,0.034014,3178.336379,11.992239,4.882531e+05,11.679274,153.619074,622.977760,91.803294,4.055341,21.873135,28.266883,5.705815,1.784807,162.456175,162.296936,13.667479,3.680811,35.689574,7.912341,8.977344,2.753125,117.413905,65.508160,-0.460370,0.410709,42.460987,115.925375,0.845991,0.431793,702.571174,302.817480,20.652438,3.998852,147.0,4.0,15.0,32.225018,0.053633,1.0,97.0,199.953250,199.953250,799.8130,0.0000,0.0000,NaN,0.013,65,1.568287,3.671722,0.000000,21.873135,1.286655,0.2426,0.07140,2,0.0000,0.1691,0.02856,0.6250,-7.383,1,2.351330e-29,1.100730e-01,3.12249,1.669840,0.413225,2.63550,0.698985,11.699300,3.12249,0.666667,0.961661,0.881326,0.257244,5.264420,1.573640,16.47190,0.527594,22.95970,0.000,1.0,1.0,0,0,0,0,-114.710914,506.473179,-272.734698,497.529038
7844,130755807,-69.036392,363.402466,16.466672,1.489275,68.780504,3.818545,2.326328,71.215874,18.477200,15.121149,14.085974,1.175512,0.052632,3025.366219,5.549463,9.152266e+05,5.798803,302.517634,432.438858,26.261461,1.429467,295.622920,333.272992,10.607947,3.192929,76.624077,61.855298,9.208880,2.970945,14.033524,21.132148,-0.523883,0.014138,380.065039,328.717896,13.153053,3.576847,1018.956828,973.270337,5.009929,2.493254,119.965253,184.278912,2.815352,1.133398,114.0,11.0,23.0,29.287346,0.036457,5.0,55.0,7.403260,7.403260,37.0163,2.5606,1.1146,46.6108,0.136,90,1.719466,-0.907001,7.805601,295.622920,24.635242,0.2886,0.09090,0,0.1200,0.0000,0.00000,0.3890,-9.390,0,1.676440e-09,5.079140e-02,3.74433,2.913970,0.563771,2.49580,1.011080,3.306760,3.74433,0.222222,0.979527,0.790144,4.068030,2.823260,-0.000683,106.13100,0.885791,8.84614,0.000,1.0,1.0,0,0,0,0,-41.840749,136.011173,-113.020908,131.799020
7845,130762946,-135.60263

In [15]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_pl

In [16]:
#A0
train_metadata['A0_max_min_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_max_mean_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_mean_flux']
train_metadata['A0_max_median_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_median_flux']
train_metadata['A0_median_mean_diff_flux'] = train_metadata['A0_median_flux'] - train_metadata['A0_mean_flux']
train_metadata['A0_median_min_diff_flux'] = train_metadata['A0_median_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_mean_min_diff_flux'] = train_metadata['A0_mean_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_minus_3_sigma'] = train_metadata['A0_mean_flux'] - 3*train_metadata['A0_std_flux']
train_metadata['A0_minus_2_sigma'] = train_metadata['A0_mean_flux'] - 2*train_metadata['A0_std_flux']
train_metadata['A0_minus_1_sigma'] = train_metadata['A0_mean_flux'] - 1*train_metadata['A0_std_flux']
train_metadata['A0_plus_1_sigma'] = train_metadata['A0_mean_flux'] + 1*train_metadata['A0_std_flux']
train_metadata['A0_plus_2_sigma'] = train_metadata['A0_mean_flux'] + 2*train_metadata['A0_std_flux']
train_metadata['A0_plus_3_sigma'] = train_metadata['A0_mean_flux'] + 3*train_metadata['A0_std_flux']
#A1
train_metadata['A1_max_min_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_max_mean_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_mean_flux']
train_metadata['A1_max_median_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_median_flux']
train_metadata['A1_median_mean_diff_flux'] = train_metadata['A1_median_flux'] - train_metadata['A1_mean_flux']
train_metadata['A1_median_min_diff_flux'] = train_metadata['A1_median_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_mean_min_diff_flux'] = train_metadata['A1_mean_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_minus_3_sigma'] = train_metadata['A1_mean_flux'] - 3*train_metadata['A1_std_flux']
train_metadata['A1_minus_2_sigma'] = train_metadata['A1_mean_flux'] - 2*train_metadata['A1_std_flux']
train_metadata['A1_minus_1_sigma'] = train_metadata['A1_mean_flux'] - 1*train_metadata['A1_std_flux']
train_metadata['A1_plus_1_sigma'] = train_metadata['A1_mean_flux'] + 1*train_metadata['A1_std_flux']
train_metadata['A1_plus_2_sigma'] = train_metadata['A1_mean_flux'] + 2*train_metadata['A1_std_flux']
train_metadata['A1_plus_3_sigma'] = train_metadata['A1_mean_flux'] + 3*train_metadata['A1_std_flux']
#A2
train_metadata['A2_max_min_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_max_mean_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_mean_flux']
train_metadata['A2_max_median_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_median_flux']
train_metadata['A2_median_mean_diff_flux'] = train_metadata['A2_median_flux'] - train_metadata['A2_mean_flux']
train_metadata['A2_median_min_diff_flux'] = train_metadata['A2_median_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_mean_min_diff_flux'] = train_metadata['A2_mean_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_minus_3_sigma'] = train_metadata['A2_mean_flux'] - 3*train_metadata['A2_std_flux']
train_metadata['A2_minus_2_sigma'] = train_metadata['A2_mean_flux'] - 2*train_metadata['A2_std_flux']
train_metadata['A2_minus_1_sigma'] = train_metadata['A2_mean_flux'] - 1*train_metadata['A2_std_flux']
train_metadata['A2_plus_1_sigma'] = train_metadata['A2_mean_flux'] + 1*train_metadata['A2_std_flux']
train_metadata['A2_plus_2_sigma'] = train_metadata['A2_mean_flux'] + 2*train_metadata['A2_std_flux']
train_metadata['A2_plus_3_sigma'] = train_metadata['A2_mean_flux'] + 3*train_metadata['A2_std_flux']
#A3
train_metadata['A3_max_min_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_max_mean_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_mean_flux']
train_metadata['A3_max_median_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_median_flux']
train_metadata['A3_median_mean_diff_flux'] = train_metadata['A3_median_flux'] - train_metadata['A3_mean_flux']
train_metadata['A3_median_min_diff_flux'] = train_metadata['A3_median_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_mean_min_diff_flux'] = train_metadata['A3_mean_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_minus_3_sigma'] = train_metadata['A3_mean_flux'] - 3*train_metadata['A3_std_flux']
train_metadata['A3_minus_2_sigma'] = train_metadata['A3_mean_flux'] - 2*train_metadata['A3_std_flux']
train_metadata['A3_minus_1_sigma'] = train_metadata['A3_mean_flux'] - 1*train_metadata['A3_std_flux']
train_metadata['A3_plus_1_sigma'] = train_metadata['A3_mean_flux'] + 1*train_metadata['A3_std_flux']
train_metadata['A3_plus_2_sigma'] = train_metadata['A3_mean_flux'] + 2*train_metadata['A3_std_flux']
train_metadata['A3_plus_3_sigma'] = train_metadata['A3_mean_flux'] + 3*train_metadata['A3_std_flux']
#A4
train_metadata['A4_max_min_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_max_mean_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_mean_flux']
train_metadata['A4_max_median_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_median_flux']
train_metadata['A4_median_mean_diff_flux'] = train_metadata['A4_median_flux'] - train_metadata['A4_mean_flux']
train_metadata['A4_median_min_diff_flux'] = train_metadata['A4_median_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_mean_min_diff_flux'] = train_metadata['A4_mean_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_minus_3_sigma'] = train_metadata['A4_mean_flux'] - 3*train_metadata['A4_std_flux']
train_metadata['A4_minus_2_sigma'] = train_metadata['A4_mean_flux'] - 2*train_metadata['A4_std_flux']
train_metadata['A4_minus_1_sigma'] = train_metadata['A4_mean_flux'] - 1*train_metadata['A4_std_flux']
train_metadata['A4_plus_1_sigma'] = train_metadata['A4_mean_flux'] + 1*train_metadata['A4_std_flux']
train_metadata['A4_plus_2_sigma'] = train_metadata['A4_mean_flux'] + 2*train_metadata['A4_std_flux']
train_metadata['A4_plus_3_sigma'] = train_metadata['A4_mean_flux'] + 3*train_metadata['A4_std_flux']
#A5
train_metadata['A5_max_min_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_max_mean_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_mean_flux']
train_metadata['A5_max_median_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_median_flux']
train_metadata['A5_median_mean_diff_flux'] = train_metadata['A5_median_flux'] - train_metadata['A5_mean_flux']
train_metadata['A5_median_min_diff_flux'] = train_metadata['A5_median_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_mean_min_diff_flux'] = train_metadata['A5_mean_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_minus_3_sigma'] = train_metadata['A5_mean_flux'] - 3*train_metadata['A5_std_flux']
train_metadata['A5_minus_2_sigma'] = train_metadata['A5_mean_flux'] - 2*train_metadata['A5_std_flux']
train_metadata['A5_minus_1_sigma'] = train_metadata['A5_mean_flux'] - 1*train_metadata['A5_std_flux']
train_metadata['A5_plus_1_sigma'] = train_metadata['A5_mean_flux'] + 1*train_metadata['A5_std_flux']
train_metadata['A5_plus_2_sigma'] = train_metadata['A5_mean_flux'] + 2*train_metadata['A5_std_flux']
train_metadata['A5_plus_3_sigma'] = train_metadata['A5_mean_flux'] + 3*train_metadata['A5_std_flux']

In [17]:
#A0
test_metadata['A0_max_min_diff_flux'] = test_metadata['A0_max_flux'] - test_metadata['A0_min_flux']
test_metadata['A0_max_mean_diff_flux'] = test_metadata['A0_max_flux'] - test_metadata['A0_mean_flux']
test_metadata['A0_max_median_diff_flux'] = test_metadata['A0_max_flux'] - test_metadata['A0_median_flux']
test_metadata['A0_median_mean_diff_flux'] = test_metadata['A0_median_flux'] - test_metadata['A0_mean_flux']
test_metadata['A0_median_min_diff_flux'] = test_metadata['A0_median_flux'] - test_metadata['A0_min_flux']
test_metadata['A0_mean_min_diff_flux'] = test_metadata['A0_mean_flux'] - test_metadata['A0_min_flux']
test_metadata['A0_minus_3_sigma'] = test_metadata['A0_mean_flux'] - 3*test_metadata['A0_std_flux']
test_metadata['A0_minus_2_sigma'] = test_metadata['A0_mean_flux'] - 2*test_metadata['A0_std_flux']
test_metadata['A0_minus_1_sigma'] = test_metadata['A0_mean_flux'] - 1*test_metadata['A0_std_flux']
test_metadata['A0_plus_1_sigma'] = test_metadata['A0_mean_flux'] + 1*test_metadata['A0_std_flux']
test_metadata['A0_plus_2_sigma'] = test_metadata['A0_mean_flux'] + 2*test_metadata['A0_std_flux']
test_metadata['A0_plus_3_sigma'] = test_metadata['A0_mean_flux'] + 3*test_metadata['A0_std_flux']
#A1
test_metadata['A1_max_min_diff_flux'] = test_metadata['A1_max_flux'] - test_metadata['A1_min_flux']
test_metadata['A1_max_mean_diff_flux'] = test_metadata['A1_max_flux'] - test_metadata['A1_mean_flux']
test_metadata['A1_max_median_diff_flux'] = test_metadata['A1_max_flux'] - test_metadata['A1_median_flux']
test_metadata['A1_median_mean_diff_flux'] = test_metadata['A1_median_flux'] - test_metadata['A1_mean_flux']
test_metadata['A1_median_min_diff_flux'] = test_metadata['A1_median_flux'] - test_metadata['A1_min_flux']
test_metadata['A1_mean_min_diff_flux'] = test_metadata['A1_mean_flux'] - test_metadata['A1_min_flux']
test_metadata['A1_minus_3_sigma'] = test_metadata['A1_mean_flux'] - 3*test_metadata['A1_std_flux']
test_metadata['A1_minus_2_sigma'] = test_metadata['A1_mean_flux'] - 2*test_metadata['A1_std_flux']
test_metadata['A1_minus_1_sigma'] = test_metadata['A1_mean_flux'] - 1*test_metadata['A1_std_flux']
test_metadata['A1_plus_1_sigma'] = test_metadata['A1_mean_flux'] + 1*test_metadata['A1_std_flux']
test_metadata['A1_plus_2_sigma'] = test_metadata['A1_mean_flux'] + 2*test_metadata['A1_std_flux']
test_metadata['A1_plus_3_sigma'] = test_metadata['A1_mean_flux'] + 3*test_metadata['A1_std_flux']
#A2
test_metadata['A2_max_min_diff_flux'] = test_metadata['A2_max_flux'] - test_metadata['A2_min_flux']
test_metadata['A2_max_mean_diff_flux'] = test_metadata['A2_max_flux'] - test_metadata['A2_mean_flux']
test_metadata['A2_max_median_diff_flux'] = test_metadata['A2_max_flux'] - test_metadata['A2_median_flux']
test_metadata['A2_median_mean_diff_flux'] = test_metadata['A2_median_flux'] - test_metadata['A2_mean_flux']
test_metadata['A2_median_min_diff_flux'] = test_metadata['A2_median_flux'] - test_metadata['A2_min_flux']
test_metadata['A2_mean_min_diff_flux'] = test_metadata['A2_mean_flux'] - test_metadata['A2_min_flux']
test_metadata['A2_minus_3_sigma'] = test_metadata['A2_mean_flux'] - 3*test_metadata['A2_std_flux']
test_metadata['A2_minus_2_sigma'] = test_metadata['A2_mean_flux'] - 2*test_metadata['A2_std_flux']
test_metadata['A2_minus_1_sigma'] = test_metadata['A2_mean_flux'] - 1*test_metadata['A2_std_flux']
test_metadata['A2_plus_1_sigma'] = test_metadata['A2_mean_flux'] + 1*test_metadata['A2_std_flux']
test_metadata['A2_plus_2_sigma'] = test_metadata['A2_mean_flux'] + 2*test_metadata['A2_std_flux']
test_metadata['A2_plus_3_sigma'] = test_metadata['A2_mean_flux'] + 3*test_metadata['A2_std_flux']
#A3
test_metadata['A3_max_min_diff_flux'] = test_metadata['A3_max_flux'] - test_metadata['A3_min_flux']
test_metadata['A3_max_mean_diff_flux'] = test_metadata['A3_max_flux'] - test_metadata['A3_mean_flux']
test_metadata['A3_max_median_diff_flux'] = test_metadata['A3_max_flux'] - test_metadata['A3_median_flux']
test_metadata['A3_median_mean_diff_flux'] = test_metadata['A3_median_flux'] - test_metadata['A3_mean_flux']
test_metadata['A3_median_min_diff_flux'] = test_metadata['A3_median_flux'] - test_metadata['A3_min_flux']
test_metadata['A3_mean_min_diff_flux'] = test_metadata['A3_mean_flux'] - test_metadata['A3_min_flux']
test_metadata['A3_minus_3_sigma'] = test_metadata['A3_mean_flux'] - 3*test_metadata['A3_std_flux']
test_metadata['A3_minus_2_sigma'] = test_metadata['A3_mean_flux'] - 2*test_metadata['A3_std_flux']
test_metadata['A3_minus_1_sigma'] = test_metadata['A3_mean_flux'] - 1*test_metadata['A3_std_flux']
test_metadata['A3_plus_1_sigma'] = test_metadata['A3_mean_flux'] + 1*test_metadata['A3_std_flux']
test_metadata['A3_plus_2_sigma'] = test_metadata['A3_mean_flux'] + 2*test_metadata['A3_std_flux']
test_metadata['A3_plus_3_sigma'] = test_metadata['A3_mean_flux'] + 3*test_metadata['A3_std_flux']
#A4
test_metadata['A4_max_min_diff_flux'] = test_metadata['A4_max_flux'] - test_metadata['A4_min_flux']
test_metadata['A4_max_mean_diff_flux'] = test_metadata['A4_max_flux'] - test_metadata['A4_mean_flux']
test_metadata['A4_max_median_diff_flux'] = test_metadata['A4_max_flux'] - test_metadata['A4_median_flux']
test_metadata['A4_median_mean_diff_flux'] = test_metadata['A4_median_flux'] - test_metadata['A4_mean_flux']
test_metadata['A4_median_min_diff_flux'] = test_metadata['A4_median_flux'] - test_metadata['A4_min_flux']
test_metadata['A4_mean_min_diff_flux'] = test_metadata['A4_mean_flux'] - test_metadata['A4_min_flux']
test_metadata['A4_minus_3_sigma'] = test_metadata['A4_mean_flux'] - 3*test_metadata['A4_std_flux']
test_metadata['A4_minus_2_sigma'] = test_metadata['A4_mean_flux'] - 2*test_metadata['A4_std_flux']
test_metadata['A4_minus_1_sigma'] = test_metadata['A4_mean_flux'] - 1*test_metadata['A4_std_flux']
test_metadata['A4_plus_1_sigma'] = test_metadata['A4_mean_flux'] + 1*test_metadata['A4_std_flux']
test_metadata['A4_plus_2_sigma'] = test_metadata['A4_mean_flux'] + 2*test_metadata['A4_std_flux']
test_metadata['A4_plus_3_sigma'] = test_metadata['A4_mean_flux'] + 3*test_metadata['A4_std_flux']
#A5
test_metadata['A5_max_min_diff_flux'] = test_metadata['A5_max_flux'] - test_metadata['A5_min_flux']
test_metadata['A5_max_mean_diff_flux'] = test_metadata['A5_max_flux'] - test_metadata['A5_mean_flux']
test_metadata['A5_max_median_diff_flux'] = test_metadata['A5_max_flux'] - test_metadata['A5_median_flux']
test_metadata['A5_median_mean_diff_flux'] = test_metadata['A5_median_flux'] - test_metadata['A5_mean_flux']
test_metadata['A5_median_min_diff_flux'] = test_metadata['A5_median_flux'] - test_metadata['A5_min_flux']
test_metadata['A5_mean_min_diff_flux'] = test_metadata['A5_mean_flux'] - test_metadata['A5_min_flux']
test_metadata['A5_minus_3_sigma'] = test_metadata['A5_mean_flux'] - 3*test_metadata['A5_std_flux']
test_metadata['A5_minus_2_sigma'] = test_metadata['A5_mean_flux'] - 2*test_metadata['A5_std_flux']
test_metadata['A5_minus_1_sigma'] = test_metadata['A5_mean_flux'] - 1*test_metadata['A5_std_flux']
test_metadata['A5_plus_1_sigma'] = test_metadata['A5_mean_flux'] + 1*test_metadata['A5_std_flux']
test_metadata['A5_plus_2_sigma'] = test_metadata['A5_mean_flux'] + 2*test_metadata['A5_std_flux']
test_metadata['A5_plus_3_sigma'] = test_metadata['A5_mean_flux'] + 3*test_metadata['A5_std_flux']

In [18]:
#F1
train_metadata['diff_A5_A4_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] - train_metadata['A4_max_min_diff_flux']
train_metadata['diff_A5_A3_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] - train_metadata['A3_max_min_diff_flux']
train_metadata['diff_A5_A2_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] - train_metadata['A2_max_min_diff_flux']
train_metadata['diff_A5_A1_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] - train_metadata['A1_max_min_diff_flux']
train_metadata['diff_A5_A0_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] - train_metadata['A0_max_min_diff_flux']
train_metadata['diff_A4_A3_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] - train_metadata['A3_max_min_diff_flux']
train_metadata['diff_A4_A2_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] - train_metadata['A2_max_min_diff_flux']
train_metadata['diff_A4_A1_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] - train_metadata['A1_max_min_diff_flux']
train_metadata['diff_A4_A0_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] - train_metadata['A0_max_min_diff_flux']
train_metadata['diff_A3_A2_max_min_flux'] = train_metadata['A3_max_min_diff_flux'] - train_metadata['A2_max_min_diff_flux']
train_metadata['diff_A3_A1_max_min_flux'] = train_metadata['A3_max_min_diff_flux'] - train_metadata['A1_max_min_diff_flux']
train_metadata['diff_A3_A0_max_min_flux'] = train_metadata['A3_max_min_diff_flux'] - train_metadata['A0_max_min_diff_flux']
train_metadata['diff_A2_A1_max_min_flux'] = train_metadata['A2_max_min_diff_flux'] - train_metadata['A1_max_min_diff_flux']
train_metadata['diff_A2_A0_max_min_flux'] = train_metadata['A2_max_min_diff_flux'] - train_metadata['A0_max_min_diff_flux']
train_metadata['diff_A1_A0_max_min_flux'] = train_metadata['A1_max_min_diff_flux'] - train_metadata['A0_max_min_diff_flux']
#F2
train_metadata['diff_A5_A4_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] - train_metadata['A4_max_mean_diff_flux']
train_metadata['diff_A5_A3_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] - train_metadata['A3_max_mean_diff_flux']
train_metadata['diff_A5_A2_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] - train_metadata['A2_max_mean_diff_flux']
train_metadata['diff_A5_A1_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] - train_metadata['A1_max_mean_diff_flux']
train_metadata['diff_A5_A0_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] - train_metadata['A0_max_mean_diff_flux']
train_metadata['diff_A4_A3_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] - train_metadata['A3_max_mean_diff_flux']
train_metadata['diff_A4_A2_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] - train_metadata['A2_max_mean_diff_flux']
train_metadata['diff_A4_A1_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] - train_metadata['A1_max_mean_diff_flux']
train_metadata['diff_A4_A0_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] - train_metadata['A0_max_mean_diff_flux']
train_metadata['diff_A3_A2_max_mean_flux'] = train_metadata['A3_max_mean_diff_flux'] - train_metadata['A2_max_mean_diff_flux']
train_metadata['diff_A3_A1_max_mean_flux'] = train_metadata['A3_max_mean_diff_flux'] - train_metadata['A1_max_mean_diff_flux']
train_metadata['diff_A3_A0_max_mean_flux'] = train_metadata['A3_max_mean_diff_flux'] - train_metadata['A0_max_mean_diff_flux']
train_metadata['diff_A2_A1_max_mean_flux'] = train_metadata['A2_max_mean_diff_flux'] - train_metadata['A1_max_mean_diff_flux']
train_metadata['diff_A2_A0_max_mean_flux'] = train_metadata['A2_max_mean_diff_flux'] - train_metadata['A0_max_mean_diff_flux']
train_metadata['diff_A1_A0_max_mean_flux'] = train_metadata['A1_max_mean_diff_flux'] - train_metadata['A0_max_mean_diff_flux']
#F3
train_metadata['diff_A5_A4_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] - train_metadata['A4_max_median_diff_flux']
train_metadata['diff_A5_A3_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] - train_metadata['A3_max_median_diff_flux']
train_metadata['diff_A5_A2_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] - train_metadata['A2_max_median_diff_flux']
train_metadata['diff_A5_A1_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] - train_metadata['A1_max_median_diff_flux']
train_metadata['diff_A5_A0_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] - train_metadata['A0_max_median_diff_flux']
train_metadata['diff_A4_A3_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] - train_metadata['A3_max_median_diff_flux']
train_metadata['diff_A4_A2_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] - train_metadata['A2_max_median_diff_flux']
train_metadata['diff_A4_A1_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] - train_metadata['A1_max_median_diff_flux']
train_metadata['diff_A4_A0_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] - train_metadata['A0_max_median_diff_flux']
train_metadata['diff_A3_A2_max_median_flux'] = train_metadata['A3_max_median_diff_flux'] - train_metadata['A2_max_median_diff_flux']
train_metadata['diff_A3_A1_max_median_flux'] = train_metadata['A3_max_median_diff_flux'] - train_metadata['A1_max_median_diff_flux']
train_metadata['diff_A3_A0_max_median_flux'] = train_metadata['A3_max_median_diff_flux'] - train_metadata['A0_max_median_diff_flux']
train_metadata['diff_A2_A1_max_median_flux'] = train_metadata['A2_max_median_diff_flux'] - train_metadata['A1_max_median_diff_flux']
train_metadata['diff_A2_A0_max_median_flux'] = train_metadata['A2_max_median_diff_flux'] - train_metadata['A0_max_median_diff_flux']
train_metadata['diff_A1_A0_max_median_flux'] = train_metadata['A1_max_median_diff_flux'] - train_metadata['A0_max_median_diff_flux']
#F4
train_metadata['diff_A5_A4_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] - train_metadata['A4_median_mean_diff_flux']
train_metadata['diff_A5_A3_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] - train_metadata['A3_median_mean_diff_flux']
train_metadata['diff_A5_A2_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] - train_metadata['A2_median_mean_diff_flux']
train_metadata['diff_A5_A1_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] - train_metadata['A1_median_mean_diff_flux']
train_metadata['diff_A5_A0_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] - train_metadata['A0_median_mean_diff_flux']
train_metadata['diff_A4_A3_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] - train_metadata['A3_median_mean_diff_flux']
train_metadata['diff_A4_A2_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] - train_metadata['A2_median_mean_diff_flux']
train_metadata['diff_A4_A1_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] - train_metadata['A1_median_mean_diff_flux']
train_metadata['diff_A4_A0_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] - train_metadata['A0_median_mean_diff_flux']
train_metadata['diff_A3_A2_median_mean_flux'] = train_metadata['A3_median_mean_diff_flux'] - train_metadata['A2_median_mean_diff_flux']
train_metadata['diff_A3_A1_median_mean_flux'] = train_metadata['A3_median_mean_diff_flux'] - train_metadata['A1_median_mean_diff_flux']
train_metadata['diff_A3_A0_median_mean_flux'] = train_metadata['A3_median_mean_diff_flux'] - train_metadata['A0_median_mean_diff_flux']
train_metadata['diff_A2_A1_median_mean_flux'] = train_metadata['A2_median_mean_diff_flux'] - train_metadata['A1_median_mean_diff_flux']
train_metadata['diff_A2_A0_median_mean_flux'] = train_metadata['A2_median_mean_diff_flux'] - train_metadata['A0_median_mean_diff_flux']
train_metadata['diff_A1_A0_median_mean_flux'] = train_metadata['A1_median_mean_diff_flux'] - train_metadata['A0_median_mean_diff_flux']
#F5
train_metadata['diff_A5_A4_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] - train_metadata['A4_median_min_diff_flux']
train_metadata['diff_A5_A3_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] - train_metadata['A3_median_min_diff_flux']
train_metadata['diff_A5_A2_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] - train_metadata['A2_median_min_diff_flux']
train_metadata['diff_A5_A1_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] - train_metadata['A1_median_min_diff_flux']
train_metadata['diff_A5_A0_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] - train_metadata['A0_median_min_diff_flux']
train_metadata['diff_A4_A3_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] - train_metadata['A3_median_min_diff_flux']
train_metadata['diff_A4_A2_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] - train_metadata['A2_median_min_diff_flux']
train_metadata['diff_A4_A1_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] - train_metadata['A1_median_min_diff_flux']
train_metadata['diff_A4_A0_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] - train_metadata['A0_median_min_diff_flux']
train_metadata['diff_A3_A2_median_min_flux'] = train_metadata['A3_median_min_diff_flux'] - train_metadata['A2_median_min_diff_flux']
train_metadata['diff_A3_A1_median_min_flux'] = train_metadata['A3_median_min_diff_flux'] - train_metadata['A1_median_min_diff_flux']
train_metadata['diff_A3_A0_median_min_flux'] = train_metadata['A3_median_min_diff_flux'] - train_metadata['A0_median_min_diff_flux']
train_metadata['diff_A2_A1_median_min_flux'] = train_metadata['A2_median_min_diff_flux'] - train_metadata['A1_median_min_diff_flux']
train_metadata['diff_A2_A0_median_min_flux'] = train_metadata['A2_median_min_diff_flux'] - train_metadata['A0_median_min_diff_flux']
train_metadata['diff_A1_A0_median_min_flux'] = train_metadata['A1_median_min_diff_flux'] - train_metadata['A0_median_min_diff_flux']
#F6
train_metadata['diff_A5_A4_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] - train_metadata['A4_mean_min_diff_flux']
train_metadata['diff_A5_A3_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] - train_metadata['A3_mean_min_diff_flux']
train_metadata['diff_A5_A2_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] - train_metadata['A2_mean_min_diff_flux']
train_metadata['diff_A5_A1_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] - train_metadata['A1_mean_min_diff_flux']
train_metadata['diff_A5_A0_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] - train_metadata['A0_mean_min_diff_flux']
train_metadata['diff_A4_A3_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] - train_metadata['A3_mean_min_diff_flux']
train_metadata['diff_A4_A2_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] - train_metadata['A2_mean_min_diff_flux']
train_metadata['diff_A4_A1_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] - train_metadata['A1_mean_min_diff_flux']
train_metadata['diff_A4_A0_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] - train_metadata['A0_mean_min_diff_flux']
train_metadata['diff_A3_A2_mean_min_flux'] = train_metadata['A3_mean_min_diff_flux'] - train_metadata['A2_mean_min_diff_flux']
train_metadata['diff_A3_A1_mean_min_flux'] = train_metadata['A3_mean_min_diff_flux'] - train_metadata['A1_mean_min_diff_flux']
train_metadata['diff_A3_A0_mean_min_flux'] = train_metadata['A3_mean_min_diff_flux'] - train_metadata['A0_mean_min_diff_flux']
train_metadata['diff_A2_A1_mean_min_flux'] = train_metadata['A2_mean_min_diff_flux'] - train_metadata['A1_mean_min_diff_flux']
train_metadata['diff_A2_A0_mean_min_flux'] = train_metadata['A2_mean_min_diff_flux'] - train_metadata['A0_mean_min_diff_flux']
train_metadata['diff_A1_A0_mean_min_flux'] = train_metadata['A1_mean_min_diff_flux'] - train_metadata['A0_mean_min_diff_flux']

In [19]:
#F7
train_metadata['diff_A5_A4_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] - train_metadata['A4_plus_3_sigma']
train_metadata['diff_A5_A3_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] - train_metadata['A3_plus_3_sigma']
train_metadata['diff_A5_A2_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] - train_metadata['A2_plus_3_sigma']
train_metadata['diff_A5_A1_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] - train_metadata['A1_plus_3_sigma']
train_metadata['diff_A5_A0_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] - train_metadata['A0_plus_3_sigma']
train_metadata['diff_A4_A3_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] - train_metadata['A3_plus_3_sigma']
train_metadata['diff_A4_A2_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] - train_metadata['A2_plus_3_sigma']
train_metadata['diff_A4_A1_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] - train_metadata['A1_plus_3_sigma']
train_metadata['diff_A4_A0_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] - train_metadata['A0_plus_3_sigma']
train_metadata['diff_A3_A2_plus_3_sigma'] = train_metadata['A3_plus_3_sigma'] - train_metadata['A2_plus_3_sigma']
train_metadata['diff_A3_A1_plus_3_sigma'] = train_metadata['A3_plus_3_sigma'] - train_metadata['A1_plus_3_sigma']
train_metadata['diff_A3_A0_plus_3_sigma'] = train_metadata['A3_plus_3_sigma'] - train_metadata['A0_plus_3_sigma']
train_metadata['diff_A2_A1_plus_3_sigma'] = train_metadata['A2_plus_3_sigma'] - train_metadata['A1_plus_3_sigma']
train_metadata['diff_A2_A0_plus_3_sigma'] = train_metadata['A2_plus_3_sigma'] - train_metadata['A0_plus_3_sigma']
train_metadata['diff_A1_A0_plus_3_sigma'] = train_metadata['A1_plus_3_sigma'] - train_metadata['A0_plus_3_sigma']
#F8
train_metadata['diff_A5_A4_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] - train_metadata['A4_plus_2_sigma']
train_metadata['diff_A5_A3_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] - train_metadata['A3_plus_2_sigma']
train_metadata['diff_A5_A2_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] - train_metadata['A2_plus_2_sigma']
train_metadata['diff_A5_A1_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] - train_metadata['A1_plus_2_sigma']
train_metadata['diff_A5_A0_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] - train_metadata['A0_plus_2_sigma']
train_metadata['diff_A4_A3_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] - train_metadata['A3_plus_2_sigma']
train_metadata['diff_A4_A2_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] - train_metadata['A2_plus_2_sigma']
train_metadata['diff_A4_A1_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] - train_metadata['A1_plus_2_sigma']
train_metadata['diff_A4_A0_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] - train_metadata['A0_plus_2_sigma']
train_metadata['diff_A3_A2_plus_2_sigma'] = train_metadata['A3_plus_2_sigma'] - train_metadata['A2_plus_2_sigma']
train_metadata['diff_A3_A1_plus_2_sigma'] = train_metadata['A3_plus_2_sigma'] - train_metadata['A1_plus_2_sigma']
train_metadata['diff_A3_A0_plus_2_sigma'] = train_metadata['A3_plus_2_sigma'] - train_metadata['A0_plus_2_sigma']
train_metadata['diff_A2_A1_plus_2_sigma'] = train_metadata['A2_plus_2_sigma'] - train_metadata['A1_plus_2_sigma']
train_metadata['diff_A2_A0_plus_2_sigma'] = train_metadata['A2_plus_2_sigma'] - train_metadata['A0_plus_2_sigma']
train_metadata['diff_A1_A0_plus_2_sigma'] = train_metadata['A1_plus_2_sigma'] - train_metadata['A0_plus_2_sigma']
#F9
train_metadata['diff_A5_A4_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] - train_metadata['A4_plus_1_sigma']
train_metadata['diff_A5_A3_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] - train_metadata['A3_plus_1_sigma']
train_metadata['diff_A5_A2_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] - train_metadata['A2_plus_1_sigma']
train_metadata['diff_A5_A1_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] - train_metadata['A1_plus_1_sigma']
train_metadata['diff_A5_A0_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] - train_metadata['A0_plus_1_sigma']
train_metadata['diff_A4_A3_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] - train_metadata['A3_plus_1_sigma']
train_metadata['diff_A4_A2_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] - train_metadata['A2_plus_1_sigma']
train_metadata['diff_A4_A1_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] - train_metadata['A1_plus_1_sigma']
train_metadata['diff_A4_A0_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] - train_metadata['A0_plus_1_sigma']
train_metadata['diff_A3_A2_plus_1_sigma'] = train_metadata['A3_plus_1_sigma'] - train_metadata['A2_plus_1_sigma']
train_metadata['diff_A3_A1_plus_1_sigma'] = train_metadata['A3_plus_1_sigma'] - train_metadata['A1_plus_1_sigma']
train_metadata['diff_A3_A0_plus_1_sigma'] = train_metadata['A3_plus_1_sigma'] - train_metadata['A0_plus_1_sigma']
train_metadata['diff_A2_A1_plus_1_sigma'] = train_metadata['A2_plus_1_sigma'] - train_metadata['A1_plus_1_sigma']
train_metadata['diff_A2_A0_plus_1_sigma'] = train_metadata['A2_plus_1_sigma'] - train_metadata['A0_plus_1_sigma']
train_metadata['diff_A1_A0_plus_1_sigma'] = train_metadata['A1_plus_1_sigma'] - train_metadata['A0_plus_1_sigma']
#F10
train_metadata['diff_A5_A4_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] - train_metadata['A4_minus_3_sigma']
train_metadata['diff_A5_A3_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] - train_metadata['A3_minus_3_sigma']
train_metadata['diff_A5_A2_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] - train_metadata['A2_minus_3_sigma']
train_metadata['diff_A5_A1_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] - train_metadata['A1_minus_3_sigma']
train_metadata['diff_A5_A0_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] - train_metadata['A0_minus_3_sigma']
train_metadata['diff_A4_A3_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] - train_metadata['A3_minus_3_sigma']
train_metadata['diff_A4_A2_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] - train_metadata['A2_minus_3_sigma']
train_metadata['diff_A4_A1_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] - train_metadata['A1_minus_3_sigma']
train_metadata['diff_A4_A0_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] - train_metadata['A0_minus_3_sigma']
train_metadata['diff_A3_A2_minus_3_sigma'] = train_metadata['A3_minus_3_sigma'] - train_metadata['A2_minus_3_sigma']
train_metadata['diff_A3_A1_minus_3_sigma'] = train_metadata['A3_minus_3_sigma'] - train_metadata['A1_minus_3_sigma']
train_metadata['diff_A3_A0_minus_3_sigma'] = train_metadata['A3_minus_3_sigma'] - train_metadata['A0_minus_3_sigma']
train_metadata['diff_A2_A1_minus_3_sigma'] = train_metadata['A2_minus_3_sigma'] - train_metadata['A1_minus_3_sigma']
train_metadata['diff_A2_A0_minus_3_sigma'] = train_metadata['A2_minus_3_sigma'] - train_metadata['A0_minus_3_sigma']
train_metadata['diff_A1_A0_minus_3_sigma'] = train_metadata['A1_minus_3_sigma'] - train_metadata['A0_minus_3_sigma']
#F11
train_metadata['diff_A5_A4_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] - train_metadata['A4_minus_2_sigma']
train_metadata['diff_A5_A3_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] - train_metadata['A3_minus_2_sigma']
train_metadata['diff_A5_A2_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] - train_metadata['A2_minus_2_sigma']
train_metadata['diff_A5_A1_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] - train_metadata['A1_minus_2_sigma']
train_metadata['diff_A5_A0_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] - train_metadata['A0_minus_2_sigma']
train_metadata['diff_A4_A3_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] - train_metadata['A3_minus_2_sigma']
train_metadata['diff_A4_A2_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] - train_metadata['A2_minus_2_sigma']
train_metadata['diff_A4_A1_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] - train_metadata['A1_minus_2_sigma']
train_metadata['diff_A4_A0_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] - train_metadata['A0_minus_2_sigma']
train_metadata['diff_A3_A2_minus_2_sigma'] = train_metadata['A3_minus_2_sigma'] - train_metadata['A2_minus_2_sigma']
train_metadata['diff_A3_A1_minus_2_sigma'] = train_metadata['A3_minus_2_sigma'] - train_metadata['A1_minus_2_sigma']
train_metadata['diff_A3_A0_minus_2_sigma'] = train_metadata['A3_minus_2_sigma'] - train_metadata['A0_minus_2_sigma']
train_metadata['diff_A2_A1_minus_2_sigma'] = train_metadata['A2_minus_2_sigma'] - train_metadata['A1_minus_2_sigma']
train_metadata['diff_A2_A0_minus_2_sigma'] = train_metadata['A2_minus_2_sigma'] - train_metadata['A0_minus_2_sigma']
train_metadata['diff_A1_A0_minus_2_sigma'] = train_metadata['A1_minus_2_sigma'] - train_metadata['A0_minus_2_sigma']
#F12
train_metadata['diff_A5_A4_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] - train_metadata['A4_minus_1_sigma']
train_metadata['diff_A5_A3_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] - train_metadata['A3_minus_1_sigma']
train_metadata['diff_A5_A2_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] - train_metadata['A2_minus_1_sigma']
train_metadata['diff_A5_A1_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] - train_metadata['A1_minus_1_sigma']
train_metadata['diff_A5_A0_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] - train_metadata['A0_minus_1_sigma']
train_metadata['diff_A4_A3_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] - train_metadata['A3_minus_1_sigma']
train_metadata['diff_A4_A2_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] - train_metadata['A2_minus_1_sigma']
train_metadata['diff_A4_A1_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] - train_metadata['A1_minus_1_sigma']
train_metadata['diff_A4_A0_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] - train_metadata['A0_minus_1_sigma']
train_metadata['diff_A3_A2_minus_1_sigma'] = train_metadata['A3_minus_1_sigma'] - train_metadata['A2_minus_1_sigma']
train_metadata['diff_A3_A1_minus_1_sigma'] = train_metadata['A3_minus_1_sigma'] - train_metadata['A1_minus_1_sigma']
train_metadata['diff_A3_A0_minus_1_sigma'] = train_metadata['A3_minus_1_sigma'] - train_metadata['A0_minus_1_sigma']
train_metadata['diff_A2_A1_minus_1_sigma'] = train_metadata['A2_minus_1_sigma'] - train_metadata['A1_minus_1_sigma']
train_metadata['diff_A2_A0_minus_1_sigma'] = train_metadata['A2_minus_1_sigma'] - train_metadata['A0_minus_1_sigma']
train_metadata['diff_A1_A0_minus_1_sigma'] = train_metadata['A1_minus_1_sigma'] - train_metadata['A0_minus_1_sigma']

In [20]:
print(train_metadata.shape)

(7848, 485)


In [21]:
train_metadata.iloc[:,233:].head()

,A0_max_min_diff_flux,A0_max_mean_diff_flux,A0_max_median_diff_flux,A0_median_mean_diff_flux,A0_median_min_diff_flux,A0_mean_min_diff_flux,A0_minus_3_sigma,A0_minus_2_sigma,A0_minus_1_sigma,A0_plus_1_sigma,A0_plus_2_sigma,A0_plus_3_sigma,A1_max_min_diff_flux,A1_max_mean_diff_flux,A1_max_median_diff_flux,A1_median_mean_diff_flux,A1_median_min_diff_flux,A1_mean_min_diff_flux,A1_minus_3_sigma,A1_minus_2_sigma,A1_minus_1_sigma,A1_plus_1_sigma,A1_plus_2_sigma,A1_plus_3_sigma,A2_max_min_diff_flux,A2_max_mean_diff_flux,A2_max_median_diff_flux,A2_median_mean_diff_flux,A2_median_min_diff_flux,A2_mean_min_diff_flux,A2_minus_3_sigma,A2_minus_2_sigma,A2_minus_1_sigma,A2_plus_1_sigma,A2_plus_2_sigma,A2_plus_3_sigma,A3_max_min_diff_flux,A3_max_mean_diff_flux,A3_max_median_diff_flux,A3_median_mean_diff_flux,A3_median_min_diff_flux,A3_mean_min_diff_flux,A3_minus_3_sigma,A3_minus_2_sigma,A3_minus_1_sigma,A3_plus_1_sigma,A3_plus_2_sigma,A3_plus_3_sigma,A4_max_min_diff_flux,A4_max_mean_diff_flux,A4_max_median_diff_flux,A4_median_mean_diff_flux,A4_median_min_diff_flux,A4_mean_min_diff_flux,A4_minus_3_sigma,A4_minus_2_sigma,A4_minus_1_sigma,A4_plus_1_sigma,A4_plus_2_sigma,A4_plus_3_sigma,A5_max_min_diff_flux,A5_max_mean_diff_flux,A5_max_median_diff_flux,A5_median_mean_diff_flux,A5_median_min_diff_flux,A5_mean_min_diff_flux,A5_minus_3_sigma,A5_minus_2_sigma,A5_minus_1_sigma,A5_plus_1_sigma,A5_plus_2_sigma,A5_plus_3_sigma,diff_A5_A4_max_min_flux,diff_A5_A3_max_min_flux,diff_A5_A2_max_min_flux,diff_A5_A1_max_min_flux,diff_A5_A0_max_min_flux,diff_A4_A3_max_min_flux,diff_A4_A2_max_min_flux,diff_A4_A1_max_min_flux,diff_A4_A0_max_min_flux,diff_A3_A2_max_min_flux,diff_A3_A1_max_min_flux,diff_A3_A0_max_min_flux,diff_A2_A1_max_min_flux,diff_A2_A0_max_min_flux,diff_A1_A0_max_min_flux,diff_A5_A4_max_mean_flux,diff_A5_A3_max_mean_flux,diff_A5_A2_max_mean_flux,diff_A5_A1_max_mean_flux,diff_A5_A0_max_mean_flux,diff_A4_A3_max_mean_flux,diff_A4_A2_max_mean_flux,diff_A4_A1_max_mean_flux,diff_A4_A0_max_mean_flux,diff_A3_A2_max_mean_flux,diff_A3_A1_max_mean_flux,diff_A3_A0_max_mean_flux,diff_A2_A1_max_mean_flux,diff_A2_A0_max_mean_flux,diff_A1_A0_max_mean_flux,diff_A5_A4_max_median_flux,diff_A5_A3_max_median_flux,diff_A5_A2_max_median_flux,diff_A5_A1_max_median_flux,diff_A5_A0_max_median_flux,diff_A4_A3_max_median_flux,diff_A4_A2_max_median_flux,diff_A4_A1_max_median_flux,diff_A4_A0_max_median_flux,diff_A3_A2_max_median_flux,diff_A3_A1_max_median_flux,diff_A3_A0_max_median_flux,diff_A2_A1_max_median_flux,diff_A2_A0_max_median_flux,diff_A1_A0_max_median_flux,diff_A5_A4_median_mean_flux,diff_A5_A3_median_mean_flux,diff_A5_A2_median_mean_flux,diff_A5_A1_median_mean_flux,diff_A5_A0_median_mean_flux,diff_A4_A3_median_mean_flux,diff_A4_A2_median_mean_flux,diff_A4_A1_median_mean_flux,diff_A4_A0_median_mean_flux,diff_A3_A2_median_mean_flux,diff_A3_A1_median_mean_flux,diff_A3_A0_median_mean_flux,diff_A2_A1_median_mean_flux,diff_A2_A0_median_mean_flux,diff_A1_A0_median_mean_flux,diff_A5_A4_median_min_flux,diff_A5_A3_median_min_flux,diff_A5_A2_median_min_flux,diff_A5_A1_median_min_flux,diff_A5_A0_median_min_flux,diff_A4_A3_median_min_flux,diff_A4_A2_median_min_flux,diff_A4_A1_median_min_flux,diff_A4_A0_median_min_flux,diff_A3_A2_median_min_flux,diff_A3_A1_median_min_flux,diff_A3_A0_median_min_flux,diff_A2_A1_median_min_flux,diff_A2_A0_median_min_flux,diff_A1_A0_median_min_flux,diff_A5_A4_mean_min_flux,diff_A5_A3_mean_min_flux,diff_A5_A2_mean_min_flux,diff_A5_A1_mean_min_flux,diff_A5_A0_mean_min_flux,diff_A4_A3_mean_min_flux,diff_A4_A2_mean_min_flux,diff_A4_A1_mean_min_flux,diff_A4_A0_mean_min_flux,diff_A3_A2_mean_min_flux,diff_A3_A1_mean_min_flux,diff_A3_A0_mean_min_flux,diff_A2_A1_mean_min_flux,diff_A2_A0_mean_min_flux,diff_A1_A0_mean_min_flux,diff_A5_A4_plus_3_sigma,diff_A5_A3_plus_3_sigma,diff_A5_A2_plus_3_sigma,diff_A5_A1_plus_3_sigma,diff_A5_A0_plus_3_sigma,diff_A4_A3_plus_3_sigma,diff_A4_A2_plus_3_sigma,diff_A4_A1_plus_3_sigma,diff_A4_A0_plus_3_sigma,diff_A3_A2_plus_3_sig

In [22]:
print(train_metadata[['object_id'] + list(train_metadata.columns)[233:]].shape)

(7848, 253)


In [23]:
print(train_metadata_kaggle.shape)

(7848, 104)


In [24]:
test_id = test_metadata_kaggle['object_id']

In [25]:
def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

In [26]:
train_metadata[list(train_metadata.columns)[-180:]].head()

,diff_A5_A4_max_min_flux,diff_A5_A3_max_min_flux,diff_A5_A2_max_min_flux,diff_A5_A1_max_min_flux,diff_A5_A0_max_min_flux,diff_A4_A3_max_min_flux,diff_A4_A2_max_min_flux,diff_A4_A1_max_min_flux,diff_A4_A0_max_min_flux,diff_A3_A2_max_min_flux,diff_A3_A1_max_min_flux,diff_A3_A0_max_min_flux,diff_A2_A1_max_min_flux,diff_A2_A0_max_min_flux,diff_A1_A0_max_min_flux,diff_A5_A4_max_mean_flux,diff_A5_A3_max_mean_flux,diff_A5_A2_max_mean_flux,diff_A5_A1_max_mean_flux,diff_A5_A0_max_mean_flux,diff_A4_A3_max_mean_flux,diff_A4_A2_max_mean_flux,diff_A4_A1_max_mean_flux,diff_A4_A0_max_mean_flux,diff_A3_A2_max_mean_flux,diff_A3_A1_max_mean_flux,diff_A3_A0_max_mean_flux,diff_A2_A1_max_mean_flux,diff_A2_A0_max_mean_flux,diff_A1_A0_max_mean_flux,diff_A5_A4_max_median_flux,diff_A5_A3_max_median_flux,diff_A5_A2_max_median_flux,diff_A5_A1_max_median_flux,diff_A5_A0_max_median_flux,diff_A4_A3_max_median_flux,diff_A4_A2_max_median_flux,diff_A4_A1_max_median_flux,diff_A4_A0_max_median_flux,diff_A3_A2_max_median_flux,diff_A3_A1_max_median_flux,diff_A3_A0_max_median_flux,diff_A2_A1_max_median_flux,diff_A2_A0_max_median_flux,diff_A1_A0_max_median_flux,diff_A5_A4_median_mean_flux,diff_A5_A3_median_mean_flux,diff_A5_A2_median_mean_flux,diff_A5_A1_median_mean_flux,diff_A5_A0_median_mean_flux,diff_A4_A3_median_mean_flux,diff_A4_A2_median_mean_flux,diff_A4_A1_median_mean_flux,diff_A4_A0_median_mean_flux,diff_A3_A2_median_mean_flux,diff_A3_A1_median_mean_flux,diff_A3_A0_median_mean_flux,diff_A2_A1_median_mean_flux,diff_A2_A0_median_mean_flux,diff_A1_A0_median_mean_flux,diff_A5_A4_median_min_flux,diff_A5_A3_median_min_flux,diff_A5_A2_median_min_flux,diff_A5_A1_median_min_flux,diff_A5_A0_median_min_flux,diff_A4_A3_median_min_flux,diff_A4_A2_median_min_flux,diff_A4_A1_median_min_flux,diff_A4_A0_median_min_flux,diff_A3_A2_median_min_flux,diff_A3_A1_median_min_flux,diff_A3_A0_median_min_flux,diff_A2_A1_median_min_flux,diff_A2_A0_median_min_flux,diff_A1_A0_median_min_flux,diff_A5_A4_mean_min_flux,diff_A5_A3_mean_min_flux,diff_A5_A2_mean_min_flux,diff_A5_A1_mean_min_flux,diff_A5_A0_mean_min_flux,diff_A4_A3_mean_min_flux,diff_A4_A2_mean_min_flux,diff_A4_A1_mean_min_flux,diff_A4_A0_mean_min_flux,diff_A3_A2_mean_min_flux,diff_A3_A1_mean_min_flux,diff_A3_A0_mean_min_flux,diff_A2_A1_mean_min_flux,diff_A2_A0_mean_min_flux,diff_A1_A0_mean_min_flux,diff_A5_A4_plus_3_sigma,diff_A5_A3_plus_3_sigma,diff_A5_A2_plus_3_sigma,diff_A5_A1_plus_3_sigma,diff_A5_A0_plus_3_sigma,diff_A4_A3_plus_3_sigma,diff_A4_A2_plus_3_sigma,diff_A4_A1_plus_3_sigma,diff_A4_A0_plus_3_sigma,diff_A3_A2_plus_3_sigma,diff_A3_A1_plus_3_sigma,diff_A3_A0_plus_3_sigma,diff_A2_A1_plus_3_sigma,diff_A2_A0_plus_3_sigma,diff_A1_A0_plus_3_sigma,diff_A5_A4_plus_2_sigma,diff_A5_A3_plus_2_sigma,diff_A5_A2_plus_2_sigma,diff_A5_A1_plus_2_sigma,diff_A5_A0_plus_2_sigma,diff_A4_A3_plus_2_sigma,diff_A4_A2_plus_2_sigma,diff_A4_A1_plus_2_sigma,diff_A4_A0_plus_2_sigma,diff_A3_A2_plus_2_sigma,diff_A3_A1_plus_2_sigma,diff_A3_A0_plus_2_sigma,diff_A2_A1_plus_2_sigma,diff_A2_A0_plus_2_sigma,diff_A1_A0_plus_2_sigma,diff_A5_A4_plus_1_sigma,diff_A5_A3_plus_1_sigma,diff_A5_A2_plus_1_sigma,diff_A5_A1_plus_1_sigma,diff_A5_A0_plus_1_sigma,diff_A4_A3_plus_1_sigma,diff_A4_A2_plus_1_sigma,diff_A4_A1_plus_1_sigma,diff_A4_A0_plus_1_sigma,diff_A3_A2_plus_1_sigma,diff_A3_A1_plus_1_sigma,diff_A3_A0_plus_1_sigma,diff_A2_A1_plus_1_sigma,diff_A2_A0_plus_1_sigma,diff_A1_A0_plus_1_sigma,diff_A5_A4_minus_3_sigma,diff_A5_A3_minus_3_sigma,diff_A5_A2_minus_3_sigma,diff_A5_A1_minus_3_sigma,diff_A5_A0_minus_3_sigma,diff_A4_A3_minus_3_sigma,diff_A4_A2_minus_3_sigma,diff_A4_A1_minus_3_sigma,diff_A4_A0_minus_3_sigma,diff_A3_A2_minus_3_sigma,diff_A3_A1_minus_3_sigma,diff_A3_A0_minus_3_sigma,diff_A2_A1_minus_3_sigma,diff_A2_A0_minus_3_sigma,diff_A1_A0_minus_3_sigma,diff_A5_A4_minus_2_sigma,diff_A5_A3_minus_2_sigma,diff_A5_A2_minus_2_sigma,diff_A5_A1_minus_2_sigma,diff_A5_A0_minus_2_sigma,diff_A4_A3_minus_2_sigma,diff_A4_A2_minus_2_sigma,diff_A4_A1_minus_2_sigma,diff

In [27]:
used_columns1 = ['diff_A5_A4_max_min_flux','diff_A2_A1_max_min_flux','diff_A3_A2_median_min_flux','diff_A5_A4_max_median_flux',
                 'diff_A4_A3_max_median_flux','diff_A2_A0_median_min_flux','diff_A4_A3_max_mean_flux','diff_A5_A2_max_mean_flux',
                 'diff_A5_A3_max_mean_flux','diff_A4_A0_median_mean_flux','diff_A5_A4_max_mean_flux','diff_A2_A1_max_median_flux']

In [28]:
used_columns2 = ['diff_A5_A2_max_median_flux','diff_A5_A4_median_min_flux','diff_A4_A0_median_min_flux',
                 'diff_A4_A1_max_median_flux','diff_A4_A2_max_median_flux',
                 'diff_A5_A4_minus_1_sigma','diff_A5_A3_median_min_flux','diff_A5_A3_max_median_flux',
                 'diff_A3_A1_minus_1_sigma','diff_A3_A0_median_min_flux','diff_A3_A0_plus_1_sigma',
                 'diff_A1_A0_median_min_flux','diff_A4_A2_mean_min_flux']

In [29]:
used_columns3 = ['diff_A5_A1_plus_1_sigma','diff_A4_A1_median_mean_flux','diff_A3_A2_max_median_flux',
                 'diff_A5_A1_median_mean_flux']

In [30]:
#used_column4 = ['diff_A5_A2_median_min_flux','diff_A1_A0_minus_1_sigma','diff_A5_A1_max_min_flux']

In [31]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()
temp = temp.merge(train_metadata[['object_id'] + used_columns1 + used_columns2 + used_columns3  ],on = 'object_id',how = 'left')
print(temp.shape)
#temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'max_depth': 7, 
'n_estimators': 1000, 
'subsample_freq': 2, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 10,
'min_child_weight': 200.0, 
#'min_child_weight': 100.0, 
'min_split_gain': 0.1, 
'num_leaves': 7, 
#'reg_alpha': 0.1,
'reg_alpha': 0.0, 
'reg_lambda': 0.00023, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

(7848, 133)
0 0.5850923663080114
1 0.5692154712572354
2 0.6272241124851677
3 0.5577627345598588
4 0.5726780345924121
MULTI WEIGHTED LOG LOSS : 0.58244 
CPU times: user 9min 43s, sys: 1.14 s, total: 9min 45s
Wall time: 1min 17s


In [32]:
#ALl
"""(7848, 284)
0 0.5950095790505551
1 0.5795815998151366
2 0.6511405032998436
3 0.57332238424385
4 0.5766957324029327
MULTI WEIGHTED LOG LOSS : 0.59520 
CPU times: user 10min 47s, sys: 904 ms, total: 10min 48s
Wall time: 1min 22s"""

'(7848, 284)\n0 0.5950095790505551\n1 0.5795815998151366\n2 0.6511405032998436\n3 0.57332238424385\n4 0.5766957324029327\nMULTI WEIGHTED LOG LOSS : 0.59520 \nCPU times: user 10min 47s, sys: 904 ms, total: 10min 48s\nWall time: 1min 22s'

In [33]:
#9
"""(7848, 125)
0 0.5833171909484508
1 0.5820517834348543
2 0.6340239749646532
3 0.5676141411296222
4 0.5824877321364343
MULTI WEIGHTED LOG LOSS : 0.58991 
CPU times: user 10min 40s, sys: 1.58 s, total: 10min 42s
Wall time: 1min 27s"""

'(7848, 125)\n0 0.5833171909484508\n1 0.5820517834348543\n2 0.6340239749646532\n3 0.5676141411296222\n4 0.5824877321364343\nMULTI WEIGHTED LOG LOSS : 0.58991 \nCPU times: user 10min 40s, sys: 1.58 s, total: 10min 42s\nWall time: 1min 27s'

In [34]:
#13
"""
(7848, 130)
0 0.5854825033812553
1 0.5755852549415591
2 0.6340095846705749
3 0.564002726116864
4 0.5744202451980789
MULTI WEIGHTED LOG LOSS : 0.58677 
CPU times: user 10min 28s, sys: 1.26 s, total: 10min 30s
Wall time: 1min 24s
"""

'\n(7848, 130)\n0 0.5854825033812553\n1 0.5755852549415591\n2 0.6340095846705749\n3 0.564002726116864\n4 0.5744202451980789\nMULTI WEIGHTED LOG LOSS : 0.58677 \nCPU times: user 10min 28s, sys: 1.26 s, total: 10min 30s\nWall time: 1min 24s\n'

In [35]:
#Used columns1,2,3
"""
(7848, 141)
0 0.5856000358492891
1 0.5688585445229082
2 0.6343161911463394
3 0.5612304450384507
4 0.5674017299642846
MULTI WEIGHTED LOG LOSS : 0.58354 
CPU times: user 10min 30s, sys: 1.2 s, total: 10min 31s
Wall time: 1min 23s
"""

'\n(7848, 141)\n0 0.5856000358492891\n1 0.5688585445229082\n2 0.6343161911463394\n3 0.5612304450384507\n4 0.5674017299642846\nMULTI WEIGHTED LOG LOSS : 0.58354 \nCPU times: user 10min 30s, sys: 1.2 s, total: 10min 31s\nWall time: 1min 23s\n'

In [36]:
print(imp_df.shape)

(131, 3)


In [37]:
"""%%time
final_dict = {}

for column_ in list(train_metadata.columns)[233:]:
    loss_list = []
    temp = train_metadata_kaggle.copy()
    temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
    y = temp['target']
    del temp['target']
    classes = sorted(y.unique())

    class_weight = {
        c: 1 for c in classes
    }
    for c in [64, 15]:
        class_weight[c] = 2

    train_id = temp['object_id']
    del temp['object_id']
    # Compute weights
    w = y.value_counts()
    weights = {i : np.sum(w) / w[i] for i in w.index}
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
    clfs = []
    importances = pd.DataFrame()
    lgb_params = {
    'random_state':51,
    'device': 'cpu', 
    'objective': 'multiclass', 
    'num_class': 14, 
    'boosting_type': 'gbdt', 
    'n_jobs': -1, 
    'max_depth': 7, 
    'n_estimators': 1000, 
    'subsample_freq': 2, 
    'subsample_for_bin': 5000, 
    'min_data_per_group': 100, 
    'max_cat_to_onehot': 4, 
    'cat_l2': 1.0, 
    'cat_smooth': 59.5, 
    'max_cat_threshold': 32, 
    'metric_freq': 10, 
    'verbosity': -1, 
    'metric': 'multi_logloss', 
    'xgboost_dart_mode': False, 
    'uniform_drop': False, 
    'colsample_bytree': 0.5, 
    'drop_rate': 0.173, 
    'learning_rate': 0.0267, 
    'max_drop': 5, 
    'min_child_samples': 10,
    'min_child_weight': 200.0, 
    #'min_child_weight': 100.0, 
    'min_split_gain': 0.1, 
    'num_leaves': 7, 
    #'reg_alpha': 0.1,
    'reg_alpha': 0.0, 
    'reg_lambda': 0.00023, 
    'skip_drop': 0.44, 
    'subsample': 0.75}
    oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
        val_x, val_y = temp.iloc[val_], y.iloc[val_]

        clf = lgb.LGBMClassifier(**lgb_params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            eval_metric=lgb_multi_weighted_logloss,
            verbose=False,
            early_stopping_rounds=50,
            sample_weight=trn_y.map(weights)
        )
        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
        loss_list.append(loss_oof)
        #print(fold_,loss_oof)

        imp_df = pd.DataFrame()
        imp_df['feature'] = temp.columns
        imp_df['gain'] = clf.feature_importances_
        imp_df['fold'] = fold_ + 1
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        clfs.append(clf)
    print(column_,'MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
    final_dict[column_] = loss_list"""

"%%time\nfinal_dict = {}\n\nfor column_ in list(train_metadata.columns)[233:]:\n    loss_list = []\n    temp = train_metadata_kaggle.copy()\n    temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')\n    y = temp['target']\n    del temp['target']\n    classes = sorted(y.unique())\n\n    class_weight = {\n        c: 1 for c in classes\n    }\n    for c in [64, 15]:\n        class_weight[c] = 2\n\n    train_id = temp['object_id']\n    del temp['object_id']\n    # Compute weights\n    w = y.value_counts()\n    weights = {i : np.sum(w) / w[i] for i in w.index}\n    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)\n    clfs = []\n    importances = pd.DataFrame()\n    lgb_params = {\n    'random_state':51,\n    'device': 'cpu', \n    'objective': 'multiclass', \n    'num_class': 14, \n    'boosting_type': 'gbdt', \n    'n_jobs': -1, \n    'max_depth': 7, \n    'n_estimators': 1000, \n    'subsample_freq': 2, \n    'subsample_for_bin': 5000, 

In [38]:
"""
final_dict2 = pd.Series(final_dict)
final_dict2 = pd.DataFrame(final_dict2)
final_dict2['fold1'] = final_dict2[0].apply(lambda x: x[0])
final_dict2['fold2'] = final_dict2[0].apply(lambda x: x[1])
final_dict2['fold3'] = final_dict2[0].apply(lambda x: x[2])
final_dict2['fold4'] = final_dict2[0].apply(lambda x: x[3])
final_dict2['fold5'] = final_dict2[0].apply(lambda x: x[4])

del final_dict2[0]

final_dict2 = final_dict2.reset_index(drop=False)

final_dict2 = final_dict2.rename(columns={'index':'column_name'})

final_dict2['fold1_1'] = (final_dict2['fold1'] > 0.62756) * 1
final_dict2['fold2_1'] = (final_dict2['fold2'] > 0.59905) * 1
final_dict2['fold3_1'] = (final_dict2['fold3'] > 0.65624) * 1
final_dict2['fold4_1'] = (final_dict2['fold4'] > 0.60978) * 1
final_dict2['fold5_1'] = (final_dict2['fold5'] > 0.61012) * 1
final_dict2['fold_sum'] = final_dict2['fold1_1']+ final_dict2['fold2_1'] + final_dict2['fold3_1'] + final_dict2['fold4_1'] + final_dict2['fold5_1']
"""

"\nfinal_dict2 = pd.Series(final_dict)\nfinal_dict2 = pd.DataFrame(final_dict2)\nfinal_dict2['fold1'] = final_dict2[0].apply(lambda x: x[0])\nfinal_dict2['fold2'] = final_dict2[0].apply(lambda x: x[1])\nfinal_dict2['fold3'] = final_dict2[0].apply(lambda x: x[2])\nfinal_dict2['fold4'] = final_dict2[0].apply(lambda x: x[3])\nfinal_dict2['fold5'] = final_dict2[0].apply(lambda x: x[4])\n\ndel final_dict2[0]\n\nfinal_dict2 = final_dict2.reset_index(drop=False)\n\nfinal_dict2 = final_dict2.rename(columns={'index':'column_name'})\n\nfinal_dict2['fold1_1'] = (final_dict2['fold1'] > 0.62756) * 1\nfinal_dict2['fold2_1'] = (final_dict2['fold2'] > 0.59905) * 1\nfinal_dict2['fold3_1'] = (final_dict2['fold3'] > 0.65624) * 1\nfinal_dict2['fold4_1'] = (final_dict2['fold4'] > 0.60978) * 1\nfinal_dict2['fold5_1'] = (final_dict2['fold5'] > 0.61012) * 1\nfinal_dict2['fold_sum'] = final_dict2['fold1_1']+ final_dict2['fold2_1'] + final_dict2['fold3_1'] + final_dict2['fold4_1'] + final_dict2['fold5_1']\n"

In [39]:
#final_dict2[final_dict2['column_name'].isin(most_imp_ones)]
#final_dict2[final_dict2['fold_sum'] == 1]

In [40]:
#final_dict2.head()

In [41]:
#final_dict2.to_csv('final_features3.csv',index=False)

In [42]:
#modify to work with kfold
#def smoteAdataset(Xig, yig, test_size=0.2, random_state=0):
def smoteAdataset(Xig_train, yig_train, Xig_test, yig_test):
    
        
    sm=SMOTE(random_state=51)
    Xig_train_res, yig_train_res = sm.fit_sample(Xig_train, yig_train.ravel())

        
    return Xig_train_res, pd.Series(yig_train_res), Xig_test, pd.Series(yig_test)

In [43]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()

#temp = temp.merge(train_metadata[['object_id'] + most_imp_ones2 ],on = 'object_id',how = 'left')

temp = temp.merge(train_metadata[['object_id'] + used_columns1 + used_columns2 + used_columns3  ],on = 'object_id',how = 'left')


print(temp.shape)
temp.fillna(0, inplace=True)

y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'max_depth': 7, 
'n_estimators': 1000, 
'subsample_freq': 2, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 10, 
'min_child_weight': 200.0, 
'min_split_gain': 0.1, 
'num_leaves': 7, 
'reg_alpha': 0.0, 
'reg_lambda': 0.00023, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    trn_xa, trn_y, val_xa, val_y=smoteAdataset(trn_x.values, trn_y.values, val_x.values, val_y.values)
    trn_x=pd.DataFrame(data=trn_xa, columns=trn_x.columns)
    val_x=pd.DataFrame(data=val_xa, columns=val_x.columns)
    
    print(trn_x.shape,trn_y.shape,val_x.shape,val_y.shape)
    
    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

(7848, 133)
(25900, 131) (25900,) (1574, 131) (1574,)
0 0.5366530871399378
(25900, 131) (25900,) (1572, 131) (1572,)
1 0.5661960904742764
(25900, 131) (25900,) (1571, 131) (1571,)
2 0.6143916522606937
(25914, 131) (25914,) (1567, 131) (1567,)
3 0.5415093686643568
(25914, 131) (25914,) (1564, 131) (1564,)
4 0.5639682004286796
MULTI WEIGHTED LOG LOSS : 0.56444 
CPU times: user 23min 56s, sys: 1.89 s, total: 23min 58s
Wall time: 3min 21s


In [44]:
used_columns = used_columns1 + used_columns2 + used_columns3 

In [45]:
test_metadata['diff_A5_A4_max_min_flux'] = test_metadata['A5_max_min_diff_flux'] - test_metadata['A4_max_min_diff_flux']
test_metadata['diff_A2_A1_max_min_flux'] = test_metadata['A2_max_min_diff_flux'] - test_metadata['A1_max_min_diff_flux']
test_metadata['diff_A3_A2_median_min_flux'] = test_metadata['A3_median_min_diff_flux'] - test_metadata['A2_median_min_diff_flux']
test_metadata['diff_A5_A4_max_median_flux'] = test_metadata['A5_max_median_diff_flux'] - test_metadata['A4_max_median_diff_flux']
test_metadata['diff_A4_A3_max_median_flux'] = test_metadata['A4_max_median_diff_flux'] - test_metadata['A3_max_median_diff_flux']
test_metadata['diff_A2_A0_median_min_flux'] = test_metadata['A2_median_min_diff_flux'] - test_metadata['A0_median_min_diff_flux']
test_metadata['diff_A4_A3_max_mean_flux'] = test_metadata['A4_max_mean_diff_flux'] - test_metadata['A3_max_mean_diff_flux']
test_metadata['diff_A5_A2_max_mean_flux'] = test_metadata['A5_max_mean_diff_flux'] - test_metadata['A2_max_mean_diff_flux']
test_metadata['diff_A5_A3_max_mean_flux'] = test_metadata['A5_max_mean_diff_flux'] - test_metadata['A3_max_mean_diff_flux']
test_metadata['diff_A4_A0_median_mean_flux'] = test_metadata['A4_median_mean_diff_flux'] - test_metadata['A0_median_mean_diff_flux']
test_metadata['diff_A5_A4_max_mean_flux'] = test_metadata['A5_max_mean_diff_flux'] - test_metadata['A4_max_mean_diff_flux']
test_metadata['diff_A2_A1_max_median_flux'] = test_metadata['A2_max_median_diff_flux'] - test_metadata['A1_max_median_diff_flux']
test_metadata['diff_A5_A2_max_median_flux'] = test_metadata['A5_max_median_diff_flux'] - test_metadata['A2_max_median_diff_flux']
test_metadata['diff_A5_A4_median_min_flux'] = test_metadata['A5_median_min_diff_flux'] - test_metadata['A4_median_min_diff_flux']
test_metadata['diff_A4_A0_median_min_flux'] = test_metadata['A4_median_min_diff_flux'] - test_metadata['A0_median_min_diff_flux']
test_metadata['diff_A4_A1_max_median_flux'] = test_metadata['A4_max_median_diff_flux'] - test_metadata['A1_max_median_diff_flux']
test_metadata['diff_A4_A2_max_median_flux'] = test_metadata['A4_max_median_diff_flux'] - test_metadata['A2_max_median_diff_flux']
test_metadata['diff_A5_A4_minus_1_sigma'] = test_metadata['A5_minus_1_sigma'] - test_metadata['A4_minus_1_sigma']
test_metadata['diff_A5_A3_median_min_flux'] = test_metadata['A5_median_min_diff_flux'] - test_metadata['A3_median_min_diff_flux']
test_metadata['diff_A5_A3_max_median_flux'] = test_metadata['A5_max_median_diff_flux'] - test_metadata['A3_max_median_diff_flux']
test_metadata['diff_A3_A1_minus_1_sigma'] = test_metadata['A3_minus_1_sigma'] - test_metadata['A1_minus_1_sigma']
test_metadata['diff_A3_A0_median_min_flux'] = test_metadata['A3_median_min_diff_flux'] - test_metadata['A0_median_min_diff_flux']
test_metadata['diff_A3_A0_plus_1_sigma'] = test_metadata['A3_plus_1_sigma'] - test_metadata['A0_plus_1_sigma']
test_metadata['diff_A1_A0_median_min_flux'] = test_metadata['A1_median_min_diff_flux'] - test_metadata['A0_median_min_diff_flux']
test_metadata['diff_A4_A2_mean_min_flux'] = test_metadata['A4_mean_min_diff_flux'] - test_metadata['A2_mean_min_diff_flux']
test_metadata['diff_A5_A1_plus_1_sigma'] = test_metadata['A5_plus_1_sigma'] - test_metadata['A1_plus_1_sigma']
test_metadata['diff_A4_A1_median_mean_flux'] = test_metadata['A4_median_mean_diff_flux'] - test_metadata['A1_median_mean_diff_flux']
test_metadata['diff_A3_A2_max_median_flux'] = test_metadata['A3_max_median_diff_flux'] - test_metadata['A2_max_median_diff_flux']
test_metadata['diff_A5_A1_median_mean_flux'] = test_metadata['A5_median_mean_diff_flux'] - test_metadata['A1_median_mean_diff_flux']

In [46]:
print(train_metadata.shape,test_metadata.shape)

(7848, 485) (3492890, 132)


In [47]:
#train_metadata.drop([x for x in train_metadata.columns if x not in ['object_id'] + used_columns ] ,axis = 1,inplace=True)

In [48]:
#test_metadata = test_metadata[[x for x in test_metadata.columns if x in ['object_id'] + used_columns ]]

In [49]:
train_metadata_kaggle = train_metadata_kaggle.merge(train_metadata[['object_id'] + used_columns ],on = 'object_id',how = 'left')
test_metadata_kaggle = test_metadata_kaggle.merge(test_metadata[['object_id'] + used_columns ],on = 'object_id',how = 'left')

In [50]:
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 133) (3492890, 132)


In [51]:
test_metadata_kaggle.head()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF,A1_minus_3_sigma,A5_max_median_diff_flux,A5_minus_3_sigma,A5_max_mean_diff_flux,diff_A5_A4_max_min_flux,diff_A2_A1_max_min_flux,diff_A3_A2_median_min_flux,diff_A5_A4_max_median_flux,diff_A4_A3_max_median_flux,diff_A2_A0_median_min_flux,diff_A4_A3_max_mean_flux,diff_A5_A2_max_mean_flux,diff_A5_A3_max_mean_flux,diff_A4_A0_median_mean_flux,diff_A5_A4_max_mean_flux,diff_A2_A1_max_median_flux,diff_A5_A2_max_median_flux,diff_A5_A4_median_min_flux,diff_A4_A0_median_min_flux,diff_A4_A1_max_median_flux,diff_A4_A2_max_median_flux,diff_A5_A4_minus_1_sigma,diff_A5_A3_median_min_flux,diff_A5_A3_max_median_flux,diff_A3_A1_minus_1_sigma,diff_A3_A0_median_min_flux,diff_A3_A0_plus_1_sigma,diff_A1_A0_median_min_flux,diff_A4_A2_mean_min_flux,diff_A5_A1_plus_1_sigma,diff_A4_A1_median_mean_flux,diff_A3_A2_max_median_flux,diff_A5_A1_median_mean_flux
0,13,-12.680235,42.765503,3.997127,0.616561,9.149645,2.037355,0.691634,11.257108,2.461810,1.972972,1.718101,1.826388,0.157576,7806.412424,4.771625,1.896346e+05,5.396523,24.292155,55.445738,13.871398,2.282455,29.002872,37.684425,-0.247160,0.147622,92.039690,79.990817,10.567412,3.117684,164.640622,139.733762,4.512783,2.171455,300.547278,246.788411,2.951479,1.806797,408.305525,349.008202,0.094963,1.153076,324.515880,251.116361,0.072799,0.804929,330.0,15.0,32.0,4.426774,0.017885,9.0,222.0,2.350061,2.350061,119.8531,0.3193,0.0542,41.1123,0.019,1.851382,3.049709,0.337084,29.002872,0.402818,0.0000,0.07690,11.0,0.3137,0.3254,0.11770,0.7114,-1.778855,6.0,0.005627,0.528087,1.384690,-0.007597,0.610190,4.81984,0.681284,1.783390,1.384690,0.711538,0.255030,0.090550,3.499440,2.788160,1.510790,13.24940,0.124063,13.15420,1.0,1.0,1.0,1,1,1,0,-13.032169,33.023636,-29.949324,30.439033,1.203725,-1.244573,3.146350,-9.275401,3.413229,-1.906972,0.572977,9.075555,-3.747502,-7.224033,-4.320479,1.245629,8.963043,10.479126,1.824685,19.484073,18.238444,-0.365809,11.064432,-5.862173,-1.251189,

In [52]:
%%time
train_metadata_kaggle.to_csv('mydata_train_metadata.csv',index=False)
test_metadata_kaggle.to_csv('mydata_test_metadata.csv',index=False)

CPU times: user 8min 57s, sys: 4.91 s, total: 9min 2s
Wall time: 9min 2s


In [53]:
imp_df.sort_values('gain',ascending=False).head()

,feature,gain,fold
57,distmod,1683,5
55,hostgal_photoz,1534,5
54,mjd_diff_det,1445,5
61,hostgal_photoz_certain,959,5
56,hostgal_photoz_err,852,5


In [54]:
temp_test = test_metadata_kaggle.copy()

In [55]:
del temp_test['object_id']

In [56]:
temp_test.fillna(0,inplace = True)

In [57]:
print(temp.shape,temp_test.shape)

(7848, 131) (3492890, 131)


In [58]:
list(temp.columns) == list(temp_test.columns)

True

In [59]:
%%time
test_pred0 = pd.DataFrame()
test_pred1 = pd.DataFrame()
test_pred2 = pd.DataFrame()
test_pred3 = pd.DataFrame()
test_pred4 = pd.DataFrame()

list_of_df = [test_pred0,test_pred1,test_pred2,test_pred3,test_pred4]

for num,c in enumerate(clfs):
    print(num)
    for k in range(0,len(temp_test),500000):
        test_pred = pd.DataFrame(c.predict_proba(temp_test[ k:k+500000] ))
        list_of_df[num] = pd.concat([list_of_df[num],test_pred],axis=0)
        del test_pred

0
1
2
3
4
CPU times: user 3h 15min 4s, sys: 26.7 s, total: 3h 15min 30s
Wall time: 28min 40s


In [60]:
test_pred2 = pd.DataFrame()
test_pred2 = (list_of_df[0] + list_of_df[1] + list_of_df[2] + list_of_df[3] + list_of_df[4])/5

In [61]:
print(test_pred2.shape)

(3492890, 14)


In [62]:
#test_pred2 = pd.DataFrame(np.random.rand(10,14))

In [63]:
test_pred2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.000204,0.001175,0.000156,0.798503,0.052875,0.000348,0.067485,0.000044,0.000129,0.003582,0.000279,0.073651,0.000259,0.001310
1,0.000115,0.006216,0.000144,0.123530,0.032477,0.000232,0.053165,0.000130,0.000559,0.018081,0.001779,0.754942,0.000221,0.008409
2,0.000241,0.014164,0.000305,0.105437,0.085017,0.000485,0.042007,0.003200,0.000661,0.056971,0.004977,0.668282,0.000309,0.017945
3,0.000277,0.002465,0.000438,0.057449,0.019708,0.000529,0.116769,0.006490,0.000513,0.562194,0.001353,0.190731,0.000327,0.040754
4,0.000052,0.000805,0.000049,0.035035,0.191075,0.000102,0.006888,0.000023,0.000076,0.004260,0.000071,0.761422,0.000061,0.000081


In [64]:
temp_columns = ['object_id','class_6','class_15','class_16','class_42','class_52','class_53','class_62','class_64','class_65','class_67','class_88','class_90','class_92','class_95','class_99']

In [65]:
test_pred2.columns = temp_columns[1:15]

In [66]:
def getUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

In [67]:
klm = pd.DataFrame()
klm['mymean'] = test_pred2[feats].mean(axis=1)
klm['mymedian'] = test_pred2[feats].median(axis=1)
klm['mymax'] = test_pred2[feats].max(axis=1)

In [68]:
test_pred2['class_99'] = getUnknown(klm)

In [69]:
test_pred2.tail()

,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
492885,0.000325,0.363273,0.000550,0.221219,0.109320,0.000618,0.142361,0.000264,0.002660,0.022318,0.003075,0.132674,0.000411,0.000933,0.247660
492886,0.000237,0.009393,0.000318,0.039336,0.164083,0.000356,0.019227,0.433752,0.000593,0.022027,0.001058,0.306150,0.000352,0.003119,0.240091
492887,0.000723,0.021722,0.000182,0.917293,0.023962,0.000384,0.026131,0.000065,0.000206,0.001022,0.000537,0.006165,0.000249,0.001360,0.066188
492888,0.000095,0.677080,0.000187,0.148660,0.003061,0.000187,0.010582,0.124167,0.000580,0.000340,0.000213,0.034020,0.000184,0.000644,0.181482
492889,0.000287,0.083177,0.000425,0.407195,0.255836,0.000627,0.063044,0.000131,0.000787,0.021987,0.001097,0.163612,0.000856,0.000939,0.242304


In [70]:
test_pred2 = test_pred2.reset_index(drop=True)

In [71]:
print(test_pred2.shape,test_id.shape)

(3492890, 15) (3492890,)


In [72]:
test_id.tail()

3492885    130787966
3492886    130787971
3492887    130787974
3492888    130788053
3492889    130788054
Name: object_id, dtype: int64

In [73]:
test_id = test_id.reset_index(drop=True)

In [74]:
test_id.index == test_pred2.index

array([ True,  True,  True, ...,  True,  True,  True])

In [75]:
%%time
test_pred = pd.concat([test_id,test_pred2],axis=1)

CPU times: user 131 ms, sys: 132 ms, total: 263 ms
Wall time: 261 ms


In [76]:
test_pred = test_pred[temp_columns]

In [77]:
test_pred.head()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
0,13,0.000204,0.001175,0.000156,0.798503,0.052875,0.000348,0.067485,0.000044,0.000129,0.003582,0.000279,0.073651,0.000259,0.001310,0.131836
1,14,0.000115,0.006216,0.000144,0.123530,0.032477,0.000232,0.053165,0.000130,0.000559,0.018081,0.001779,0.754942,0.000221,0.008409,0.152360
2,17,0.000241,0.014164,0.000305,0.105437,0.085017,0.000485,0.042007,0.003200,0.000661,0.056971,0.004977,0.668282,0.000309,0.017945,0.186707
3,23,0.000277,0.002465,0.000438,0.057449,0.019708,0.000529,0.116769,0.006490,0.000513,0.562194,0.001353,0.190731,0.000327,0.040754,0.215626
4,34,0.000052,0.000805,0.000049,0.035035,0.191075,0.000102,0.006888,0.000023,0.000076,0.004260,0.000071,0.761422,0.000061,0.000081,0.148590


In [78]:
print(test_pred.shape)

(3492890, 16)


In [79]:
%%time
test_pred.to_csv('test_pred_27.csv',index=False)

CPU times: user 1min 45s, sys: 1.93 s, total: 1min 47s
Wall time: 1min 47s


In [ ]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_27.csv -m "Message"